<a href="https://colab.research.google.com/github/Rafsan7238/CSE474_Lab/blob/main/Lab%203/CSE474_Lab_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CSE474 Lab Task 3**

### Necessary installations and imports:

In [1]:
!pip install simpy

import random
import simpy

### Parameter Intialisation:

In [2]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 40  # Total number of customers
INTERVAL_CUSTOMERS = 6.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 5  # Max. customer patience

COUNTER_LIST = [] # List of counters
WAITING_QUEUE = 0 # No. of people waiting

### Random Customer Generation:

In [3]:
def source(env, number, interval):
    """Source generates customers randomly"""
    for index in range(number):
        c = customer(env, 'Customer%02d' % index, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

### Customer Service:

In [4]:
def customer(env, name, time_in_bank):

    global WAITING_QUEUE 
    reneged = False   # Keeps track if the customer left
    assigned_counter_no = -1   # Assigned counter for service
    waiting_time = 0   # Amount of time the customer is waiting for
    patience = random.randint(MIN_PATIENCE, MAX_PATIENCE)   # Max time before customer wishes to leave

    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    WAITING_QUEUE += 1   # Customer waits in a queue

    while True:

      ''' 
      Opens a new counter if the queue length is > 5. 
      If the counter limit is exceeded, the customer leaves. 
      ''' 
      if WAITING_QUEUE > 5: 

        opened = open_counter(env)

        if not opened:
          print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, waiting_time))
          reneged = True
          WAITING_QUEUE -= 1
          break

      '''
      Look for empty counters. 
      If found, return the counter no., leave the queue and break out of loop. 
      '''
      for counter_no in range(0, len(COUNTER_LIST)):
        if COUNTER_LIST[counter_no].count == 0:

          print('%7.4f %s: Waited %6.3f' % (env.now, name, waiting_time))
          assigned_counter_no = counter_no
          WAITING_QUEUE -= 1
          break

      '''
      If no counter is found after an iteration, increase the waiting time. 
      Increase the environment time by 1. 
      Else, break out of while loop. 
      '''
      if assigned_counter_no == -1:
        yield env.timeout(1)
        waiting_time += 1

      else:
        break

      '''
      If waiting time exceeds patience, try to open a new counter. 
      If opened, continue searching for an empty counter; else, the customer leaves.  
      '''
      if waiting_time > patience:
        print('%7.4f %s: About to give up after %6.3f' % (env.now, name, waiting_time))

        opened = open_counter(env)

        if not opened:
          print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, waiting_time))
          reneged = True
          WAITING_QUEUE -= 1
          break

      
    '''
    If not in while loop, and the customer did not leave, assign the counter according to counter no.
    The counter serves the customer for a random amount of time, and the customer leaves. 
    '''  
    if not reneged:

      counter = COUNTER_LIST[assigned_counter_no]
      req = counter.request()   # Request to be served by counter

      tib = random.expovariate(1.0 / time_in_bank)   # Amount of service time 
      yield env.timeout(tib)
      print('%7.4f %s: Served by Counter%02d' % (env.now, name, assigned_counter_no))
      counter.release(req)   # Release the request

### Open New Counter:

In [5]:
'''
Open a new counter if the counter capacity is not exceeded. 
Append the new counter to the counter list and return true. 
Else, return false. 
'''
def open_counter(env):

  global COUNTER_LIST

  if len(COUNTER_LIST) > 20:
    return False

  else:
    counter = simpy.Resource(env, capacity=1)
    print('%7.4f Counter%02d: Opened' % (env.now, len(COUNTER_LIST)))
    COUNTER_LIST.append(counter)
    return True

### Main Method

In [6]:
# Setup and start the simulation
print('Bank renege modified')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = open_counter(env)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS))
env.run()

Bank renege modified
 0.0000 Counter00: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 6.1204 Customer01: Here I am
 7.8057 Customer02: Here I am
 9.1204 Customer01: About to give up after  3.000
 9.1204 Counter01: Opened
 9.1204 Customer01: Waited  3.000
10.2118 Customer01: Served by Counter01
10.8057 Customer02: Waited  3.000
15.8073 Customer03: Here I am
15.9888 Customer04: Here I am
16.2367 Customer00: Served by Counter00
16.8073 Customer03: Waited  1.000
17.1300 Customer03: Served by Counter00
17.3823 Customer02: Served by Counter01
17.5777 Customer05: Here I am
17.5777 Customer05: Waited  0.000
17.9888 Customer04: Waited  2.000
18.9079 Customer06: Here I am
20.8639 Customer07: Here I am
20.9079 Customer06: About to give up after  2.000
20.9079 Counter02: Opened
20.9079 Customer06: Waited  2.000
23.8639 Customer07: About to give up after  3.000
23.8639 Counter03: Opened
23.8639 Customer07: Waited  3.000
24.1045 Customer05: Served by Counter00
25.1456 Custo